In [1]:
import os
import bs4
import re
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage,HumanMessage

load_dotenv()

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
os.environ['HF_API_KEY']=os.getenv('HF_API_KEY')
Groq_api_key=os.getenv('GROQ_API_KEY')

llm_model=ChatGroq(groq_api_key=Groq_api_key,model_name="llama-3.3-70b-versatile")
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11d268400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11d268310>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
Embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/homebrew/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:

loader = WebBaseLoader("https://www.ibm.com/think/topics/ai-agents")

docs = loader.load()
print(docs[0].page_content)















What Are AI Agents? | IBM



































































































                                   



  
    What are AI agents?






    


                               






























AI Agents



Welcome





Caret right

Introduction




Overview




AI agents vs AI assistants





Caret right

Agentic AI




What is agentic AI?




Why is agentic AI important?






Agentic AI vs generative AI





Caret right

AI agent development




What is AI agent development?




AgentOps







Caret right

Types of AI agents




Overview




Goal-based agent




Model-based reflex agent




Simple reflex agent




Utility-based agent









Caret right

Components




Overview





Caret right

Agentic workflows




What are agentic workflows?




Tutorial: Agentic workflows






Communication




Learning




Memory




Perception




Planning




Reasoning





Caret right

Tool calling




W

In [5]:
raw = docs[0].page_content

# removing blank lines
cleaned_text = re.sub(r"\n\s*\n+", "\n", raw)
cleaned_text= re.sub(r"[ \t]+", " ", cleaned_text)
cleaned_text = cleaned_text.strip()

print(cleaned_text[::])


What Are AI Agents? | IBM
 What are AI agents?
AI Agents
Welcome
Caret right
Introduction
Overview
AI agents vs AI assistants
Caret right
Agentic AI
What is agentic AI?
Why is agentic AI important?
Agentic AI vs generative AI
Caret right
AI agent development
What is AI agent development?
AgentOps
Caret right
Types of AI agents
Overview
Goal-based agent
Model-based reflex agent
Simple reflex agent
Utility-based agent
Caret right
Components
Overview
Caret right
Agentic workflows
What are agentic workflows?
Tutorial: Agentic workflows
Communication
Learning
Memory
Perception
Planning
Reasoning
Caret right
Tool calling
What is tool calling?
Tutorial: Ollama tool calling
Tutorial: LM Studio tool calling
Caret right
Architecture
Overview
Caret right
AI agent orchestration
What is agent orchestration?
Tutorial: Agent orchestration
Caret right
Multi-agent systems
What are multi-agent systems?
Tutorial: crewAI multi-agent call analysis
Multi-agent collaboration
Caret right
ReAct
What is ReAct?


In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)
splitted_text=text_splitter.split_text(cleaned_text)
splitted_text

['What Are AI Agents? | IBM\n What are AI agents?\nAI Agents\nWelcome\nCaret right\nIntroduction\nOverview\nAI agents vs AI assistants\nCaret right\nAgentic AI\nWhat is agentic AI?\nWhy is agentic AI important?\nAgentic AI vs generative AI\nCaret right\nAI agent development\nWhat is AI agent development?\nAgentOps\nCaret right\nTypes of AI agents\nOverview\nGoal-based agent\nModel-based reflex agent\nSimple reflex agent\nUtility-based agent\nCaret right\nComponents\nOverview\nCaret right\nAgentic workflows\nWhat are agentic workflows?\nTutorial: Agentic workflows\nCommunication\nLearning\nMemory\nPerception\nPlanning\nReasoning\nCaret right\nTool calling\nWhat is tool calling?\nTutorial: Ollama tool calling\nTutorial: LM Studio tool calling\nCaret right\nArchitecture\nOverview\nCaret right\nAI agent orchestration\nWhat is agent orchestration?\nTutorial: Agent orchestration\nCaret right\nMulti-agent systems\nWhat are multi-agent systems?\nTutorial: crewAI multi-agent call analysis\nMult

In [7]:
vector_db=Chroma.from_texts(splitted_text,embedding=Embeddings)
retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11da60f40>, search_kwargs={})

In [8]:
#Creating System Prompt
System_prompt = (
    "Answer the question using the provided context. "
    "If unsure, say you don't know. "
    "Limit your response to three concise sentences.\n\n"
    "{context}"
)

LLM_Prompt=ChatPromptTemplate.from_messages(
    [
        ("system",System_prompt),
        ('human','{input}'),
    ]
)

In [9]:
question_answer_chain=create_stuff_documents_chain(llm_model,LLM_Prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [10]:
response=rag_chain.invoke({'input':'what are ai agents?'})
response

{'input': 'what are ai agents?',
 'context': [Document(id='52836ccb-e3a4-4dfe-b54e-0ef99810c551', metadata={}, page_content='AI agents\n What are AI agents?\nFrom monolithic models to compound AI systems, discover how AI agents integrate with databases and external tools to enhance problem-solving capabilities and adaptability.\nLearn more\n Reasoning paradigms\r\nThere is not one standard architecture for building AI agents. Several paradigms exist for solving multistep problems.\nReAct (reasoning and action)\nWith the ReAct paradigm, we can instruct agents to "think" and plan after each action taken and with each tool response to decide which tool to use next. These Think-Act-Observe loops are used to solve problems step by step and iteratively improve upon responses.\nThrough the prompt structure, agents can be instructed to reason slowly and to display each "thought".4\xa0The agent\'s verbal reasoning gives insight into how responses are formulated. In this framework, agents contin

In [11]:
response['answer']

'AI agents are systems that autonomously perform tasks by designing workflows with available tools. They can encompass a wide range of functions, including decision-making, problem-solving, interacting with external environments, and performing actions. AI agents can integrate with databases and external tools to enhance problem-solving capabilities and adaptability.'

In [12]:
print(rag_chain.invoke({'input':'describe more about it'})['answer'])

AI agents are designed to perform tasks and make decisions by gathering information from various sources, learning from experiences, and using feedback mechanisms to improve their accuracy. They can be categorized into types such as utility-based agents, goal-based agents, and learning agents, each with unique capabilities and characteristics. These agents use various techniques, including internal knowledge, external databases, and communication with other agents, to achieve their goals and provide useful outputs to users.


In [13]:
print(rag_chain.invoke({'input':'What is ReWoo'})['answer'])

ReWOO (reasoning without observation) is a method that allows agents to plan upfront without relying on tool outputs, anticipating which tools to use upon receiving a user's prompt. This approach eliminates redundant tool usage and reduces token usage and computational complexity. ReWOO is a desirable approach from a human-centered perspective as it enables users to confirm the plan before it is executed.


In [14]:
print(rag_chain.invoke({'input':'describe more about it'})['answer'])

The text describes AI agents and their capabilities, including their ability to learn, reason, and interact with their environment. There are different types of agents, such as goal-based, utility-based, and learning agents, each with unique characteristics and abilities. These agents use various mechanisms, like feedback loops and knowledge bases, to improve their performance and make decisions that maximize their utility or achieve their goals.


In [15]:
#Adding Chat History

In [16]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Rewrite the user’s latest question so it can be understood on its own, "
    "without relying on prior chat history. Include any necessary context from "
    "the conversation. Do NOT answer the question—only reformulate it."
)

contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)

In [17]:
retriever_with_history=create_history_aware_retriever(llm_model,retriever,contextualize_q_prompt)
retriever_with_history

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11da60f40>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk'

In [18]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ('system',System_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)

In [19]:
question_answer_chain=create_stuff_documents_chain(llm_model,qa_prompt)
rag_chain=create_retrieval_chain(retriever_with_history,question_answer_chain)

In [20]:
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response1['answer'])

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Self-reflection in AI agents refers to their ability to evaluate and improve their own performance using feedback mechanisms. Through self-reflection, AI agents can update their internal models and adjust to user preferences, allowing them to learn and improve over time. This process enables AI agents to refine their decision-making and respond more accurately to future situations.
The provided context does not explicitly define "Self-Reflection". However, it mentions "Learning and reflection" as a process used by AI agents to improve the accuracy of their responses. This process involves using feedback mechanisms to adjust to user preferences and improve performance.


In [21]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-reflection in AI agents refers to their ability to evaluate and improve their own performance using feedback mechanisms. Through self-reflection, AI agents can update their internal models and adjust to user preferences, allowing them to learn and improve over time. This process enables AI agents to refine their decision-making and respond more accurately to future situations.', additional_kwargs={}, response_metadata={})]

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

In [23]:
store={}

def Get_Session_History(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

conversational_rag_chain= RunnableWithMessageHistory(
    rag_chain,
    Get_Session_History,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [24]:
from langsmith import uuid7

session_id = uuid7()

In [25]:
response=conversational_rag_chain.invoke({'input':"What is ReWoo?"},config={"configurable": {"session_id": session_id}},)
response

/opt/homebrew/lib/python3.10/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


{'input': 'What is ReWoo?',
 'chat_history': [],
 'context': [Document(id='fe092ccd-a365-44b8-ac24-88ac995e3eb8', metadata={}, page_content="ReWOO (reasoning without observation)\nThe ReWOO method, unlike ReAct, eliminates the dependence on tool outputs for action planning. Instead, agents plan upfront. Redundant tool usage is avoided by anticipating which tools to use upon receiving the initial prompt from the user. This approach is desirable from a human-centered perspective because the user can confirm the plan before it is executed.\nThe ReWOO workflow is made up of three modules. In the planning module, the agent anticipates its next steps given a user's prompt. The next stage entails collecting the outputs produced by calling these tools. Lastly, the agent pairs the initial plan with the tool outputs to formulate a response. This planning ahead can greatly reduce token usage and computational complexity and the repercussions of intermediate tool failure.5\n Types of AI agents"),


In [26]:
response['answer']

'ReWOO (reasoning without observation) is a method that allows AI agents to plan upfront and anticipate which tools to use, eliminating the dependence on tool outputs for action planning. This approach enables agents to avoid redundant tool usage and confirm plans with users before execution. ReWOO consists of three modules: planning, collecting tool outputs, and formulating a response.'

In [27]:
response=conversational_rag_chain.invoke({'input':"What was the previous question I asked"},config={"configurable": {"session_id": session_id}},)
response['answer']

'Your previous question was "What is ReWoo?"'

In [30]:
response=conversational_rag_chain.invoke({'input':"Tell more detailed description about it"},config={"configurable": {"session_id": session_id}},)
print(response['answer'])

ReWOO (reasoning without observation) is a novel paradigm that enables AI agents to reason and plan without relying on the outputs of tools or intermediate results. This approach is designed to improve the efficiency and effectiveness of AI agents in solving complex problems. Here is a more detailed description of ReWOO:

**Key Characteristics:**

1. **Upfront Planning**: ReWOO agents plan their actions upfront, without waiting for the outputs of tools or intermediate results. This allows them to anticipate which tools to use and in what sequence.
2. **Tool Anticipation**: ReWOO agents can anticipate which tools to use and when, reducing the need for redundant tool usage and minimizing the computational complexity of the problem-solving process.
3. **Decoupling Reasoning from Observations**: ReWOO agents decouple their reasoning from observations, allowing them to plan and reason without being influenced by intermediate results or tool outputs.
4. **Three-Stage Workflow**: The ReWOO wo

In [31]:
response=conversational_rag_chain.invoke({'input':"What is agentic ai?"},config={"configurable": {"session_id": session_id}},)
print(response['answer'])

Agentic AI refers to a type of artificial intelligence that is autonomous, proactive, and goal-oriented, with the ability to make decisions and take actions to achieve specific objectives. Agentic AI systems are designed to be self-directed and adaptive, allowing them to learn from their environment and adjust their behavior accordingly. They are characterized by their ability to perceive, reason, and act, and are often used in applications such as robotics, autonomous vehicles, and decision support systems. Agentic AI systems can be developed to have varying levels of capabilities, ranging from simple reflex agents to more advanced goal-based agents.


In [34]:
response=conversational_rag_chain.invoke({'input':"What was the first question I asked"},config={"configurable": {"session_id": session_id}},)
response['answer']

'Your first question was "What is ReWoo?"'

In [32]:
store

{UUID('019abec0-d0ee-71f0-af75-63bfc08bc756'): InMemoryChatMessageHistory(messages=[HumanMessage(content='What is ReWoo?', additional_kwargs={}, response_metadata={}), AIMessage(content='ReWOO (reasoning without observation) is a method that allows AI agents to plan upfront and anticipate which tools to use, eliminating the dependence on tool outputs for action planning. This approach enables agents to avoid redundant tool usage and confirm plans with users before execution. ReWOO consists of three modules: planning, collecting tool outputs, and formulating a response.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was the previous question I asked', additional_kwargs={}, response_metadata={}), AIMessage(content='Your previous question was "What is ReWoo?"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell more detailed description about it', additional_kwargs={}, response_metadata={}), AIMessage(content="ReWOO (reasoning without observ